In [ ]:
%pip install pyriksdagen
from lxml import etree
import progressbar
from pyparlaclarin.read import paragraph_iterator, speeches_with_name,parlaclarin_to_txt,parlaclarin_to_md
from pyriksdagen.utils import protocol_iterators, download_corpus
from pyriksdagen.metadata import load_Corpus_metadata
import pyriksdagen
# We need a parser for reading in XML data
parser = etree.XMLParser(remove_blank_text=True)
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re


In [ ]:
def train_test_valid(df):
    df_train=pd.DataFrame()
    df_test=pd.DataFrame(
    )
    df_valid=pd.DataFrame()

    for s in df["protocole"].unique():
        subset_df = df[df["protocole"] == s]
        h=hash(s)
        h_adjusted = h % (2**32 - 1)
        np.random.seed(h_adjusted)
        r = np.random.rand()
        if r <= 0.7 :
            df_train = pd.concat([df_train, subset_df], ignore_index=True)
        elif r <=0.85:
            df_test = pd.concat([df_test, subset_df], ignore_index=True)
        else:
            df_valid = pd.concat([df_valid, subset_df], ignore_index=True)
    return df_train,df_test,df_valid


In [ ]:

#download_corpus(partitions=["politicians", "records"])

In [ ]:
protocols = list(protocol_iterators(corpus_root="data/", start=1867, end=202122))

In [ ]:
protocol_in_question = protocols[10]
root = etree.parse(protocol_in_question, parser).getroot()

Dataset for pretraining models

In [ ]:
data_train=[]
data_test=[]
data_valid=[]

for i in range(len(protocols)):
  protocol_in_question = protocols[i]

  h=hash(protocol_in_question)
  h_adjusted = h % (2**32 - 1)
  np.random.seed(h_adjusted)
  r = np.random.rand()
  root = etree.parse(protocol_in_question, parser).getroot()
  element_str=paragraph_iterator(root, output="str")
  if r<=0.7:
    data_train.append({"protocole": protocol_in_question,"texte": " ".join(element_str)})
  elif r<=0.85:
    data_test.append({"protocole": protocol_in_question,"texte": " ".join(element_str)})
  else :
    data_valid.append({"protocole": protocol_in_question,"texte": " ".join(element_str)})

df_train=pd.DataFrame(data_train)
df_test=pd.DataFrame(data_test)
df_valid=pd.DataFrame(data_valid)

print(df_train.head())
#df_train,df_test = train_test_split(df,test_size=0.2,random_state=42)
df_train.to_pickle("swerick_data_random_train.pkl")
df_test.to_pickle("swerick_data_random_test.pkl")
df_valid.to_pickle("swerick_data_random_valid.pkl")

Dataset for downstreams tasks

In [ ]:
metadata =load_Corpus_metadata()
metadata =pd.DataFrame(metadata)
metadata=metadata.set_index("person_id")
print(metadata)


In [ ]:
# Party gender classficication

notes=[]
id=[]
party=[]
gender=[]
protocol=[]

for i in range(len(protocols)):
    protocol_in_question = protocols[i]
    with open(protocol_in_question,"r") as f:
        xml_content=f.read()
    soup=BeautifulSoup(xml_content,"xml")
    note_elements=soup.find_all("u")


    for note in note_elements:
        note_text = note.text.strip()
        note_text = re.sub(r'\s+',' ',note_text)
        note_text = re.sub(r'\n+',' ',note_text)
        next_element = note.get("who")
        if next_element != "unknown":
            id.append(next_element)
            notes.append(note_text)

            party_value = metadata.loc[next_element, "party"]
            if isinstance(party_value, pd.Series):
                party_values = party_value.dropna()
                if not party_values.empty:
                    party.append(party_values.iloc[0])
                else:
                    party.append(np.nan)
            else:
                party.append(party_value)

            gender_value = metadata.loc[next_element, "gender"]
            if isinstance(gender_value, pd.Series):
                gender_values = gender_value.dropna()
                if not gender_values.empty:
                    gender.append(gender_values.iloc[0])
                else:
                    gender.append(np.nan)
            else:
                gender.append(gender_value)

            protocol.append(protocol_in_question)

df = pd.DataFrame({"protocol": protocol, "Note": notes, "id": id, "party": party, "gender": gender})

df_train,df_test,df_valid=train_test_valid(df)  

In [ ]:
df_train.to_pickle("swerick_data_party_train.pkl")
df_test.to_pickle("swerick_data_party_test.pkl")
df_valid.to_pickle("swerick_data_party_valid.pkl")

In [ ]:
#NOte Seg classification



text=[]
note_seg=[]

for i in range(len(protocols)):
    protocol_in_question = protocols[i]
    with open(protocol_in_question,"r") as f:
        xml_content=f.read()
    soup=BeautifulSoup(xml_content,"xml")
    note_elements=soup.find_all("note")
    seg_elements=soup.find_all("seg")

    for note in note_elements:
        text.append(note.text.strip())
        note_seg.append("note")

    for seg in seg_elements:
        text.append(seg.text.strip())
        note_seg.append("seg")

df=pd.DataFrame({"Text" :text, "Note/seg":note_seg})
df_train_seg,df_test_seg,df_valid=train_test_valid(df)
print(df)

In [ ]:
df_train_seg,df_test_seg = train_test_split(df,test_size=0.2,random_state=42)
df_train_seg.to_pickle("swerick_data_seg_train.pkl")
df_test_seg.to_pickle("swerick_data_seg_test.pkl")